In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import docker

In [3]:
client = docker.from_env()

In [16]:
_image = "kube-webhook-certgen"
print(_image)

kube-webhook-certgen


In [17]:
old_registry = "vcr-hcm03.vngcloud.vn/k8s_images"
new_registry = "vcr.vngcloud.vn/k8s_images"

In [18]:
images = client.images.pull(f"{old_registry}/{_image}", all_tags=True)

In [19]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(image.tags[0])
    image.tag(new_image)

vcr-hcm03.vngcloud.vn/k8s_images/kube-webhook-certgen:v1.0.0


In [20]:
for image in images:
    img = image.tags[0]
    new_image = img.replace(old_registry, new_registry)
    print(f"Pushing... {new_image}")
    client.images.push(new_image)
    print(f"Pushed {new_image}\n")

Pushing... vcr.vngcloud.vn/k8s_images/kube-webhook-certgen:v1.0.0
Pushed vcr.vngcloud.vn/k8s_images/kube-webhook-certgen:v1.0.0



In [21]:
!docker image prune -a -f

Deleted Images:
untagged: vcr-hcm03.vngcloud.vn/k8s_images/kube-webhook-certgen:v1.0.0
untagged: vcr-hcm03.vngcloud.vn/k8s_images/kube-webhook-certgen@sha256:58fde0ddd7a0d1bf1483fed53e363144ae8741d8a2d6c129422e8b1b9aa0543c
untagged: vcr.vngcloud.vn/k8s_images/kube-webhook-certgen:v1.0.0
untagged: vcr.vngcloud.vn/k8s_images/kube-webhook-certgen@sha256:58fde0ddd7a0d1bf1483fed53e363144ae8741d8a2d6c129422e8b1b9aa0543c
deleted: sha256:f3b9b9fa842c4d353f869615fb18605485a59d87345f1461e78c29a487bdbbe1
deleted: sha256:5e698399e5b8c4e39238d73f4d38721cf26d6c4f556eb84698e7adc8af3fb26b
deleted: sha256:bcea74d7ae61fa36e227e9747a283de9fd7c44aafe2fc8c503bbf94705dbfa04
deleted: sha256:932da51564135c98a49a34a193d6cd363d8fa4184d957fde16c9d8527b3f3b02

Total reclaimed space: 51.53MB
